In [171]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='a50eb929-1abb-41ea-be8a-2576b05b3d2f', project_access_token='p-d8d4b116d8461d2426aec682007a26679f2cb921')
pc = project.project_context


In [172]:
import numpy as np
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup as bs
import re

In [173]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_bb2ff9ec7be94d32a0bcbbd5629362b2 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='259gjQsHpRoIqbV9mHDv8wkI-N_ijCjRh2mIsbQmrp5i',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_bb2ff9ec7be94d32a0bcbbd5629362b2.get_object(Bucket='hackathon1-donotdelete-pr-swi2lnzkhpnd0w',Key='Hackathon-data scientist.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_ds = pd.read_csv(body, header=None)
df_ds.columns = ["URL"]
df_ds.head()


body = client_bb2ff9ec7be94d32a0bcbbd5629362b2.get_object(Bucket='hackathon1-donotdelete-pr-swi2lnzkhpnd0w',Key='Hackathon-engineering.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_eng = pd.read_csv(body, header=None)
df_eng.columns = ["URL"]
df_eng.head()


body = client_bb2ff9ec7be94d32a0bcbbd5629362b2.get_object(Bucket='hackathon1-donotdelete-pr-swi2lnzkhpnd0w',Key='Hackathon-not engineering.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_neng = pd.read_csv(body, header=None)
df_neng.columns = ["URL"]
df_neng.head()



,URL
0,https://www.indeed.com/cmp/Total-Financial-Fre...
1,https://www.indeed.com/cmp/Erie-Construction/j...
2,https://www.indeed.com/viewjob?jk=946c24f2a3ae...
3,https://www.indeed.com/cmp/MaidPro-Franchising...
4,https://www.indeed.com/cmp/LTC-Consulting-Serv...


In [174]:
def selecting(df,lb):

    col_names =  ['contents', 'label']
    result  = pd.DataFrame(columns = col_names)
    
    for i in df.URL:
        x = urllib.request.urlopen(i)
        html_page = x.read()
        a = bs(html_page, 'html.parser')
        try:
            #job_title = a.find("h3", {"class":"icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title"}).contents[0]
            conts = a.find("div",{"class":"jobsearch-JobComponent-description icl-u-xs-mt--md"}).contents

        except:
            #job_title = a.find("font").contents[0]
            conts = a.find("div", id="desc").contents

        t = ""
        for i in range(len(conts)):
            try:
                t = t + conts[i].get_text()
            except:
                continue
        result = result.append({'contents': t, 'label': lb}, ignore_index=True)
        
    return result

    

In [215]:
def word_split(old_df, label):
    col_names =  ['contents', 'label']
    new_df = pd.DataFrame(columns = col_names)
    
    for i in old_df.contents:
        ls = i.split('. ')
        #ls = re.split('', i)
        #print(ls)
        for j in ls:
            #print(j)
            new_df = new_df.append({'contents':j, 'label':label}, ignore_index=True)
    return new_df

In [218]:
ds = selecting(df_ds, "Data Science")

In [219]:
new_ds = word_split(ds, "Data Science")

In [220]:
eng = selecting(df_eng, "Not Data Science")

In [221]:
new_eng = word_split(eng, "Not Data Science")

In [222]:
neng = selecting(df_neng, "Not Data Science")

In [223]:
new_neng = word_split(neng, "Not Data Science")

In [244]:
final = pd.concat([new_ds, new_eng, new_neng], ignore_index=True)

final.head()

,contents,label
0,"At Uber, we ignite opportunity by setting the ...",Data Science
1,"We take on big problems to help drivers, rider...",Data Science
2,"If you have the curiosity, passion, and collab...",Data Science
3,Come tackle fascinating and difficult problems...,Data Science
4,"To solve these problems, data scientists lever...",Data Science


In [245]:
drop_ls = []
for i in range(len(final.contents)):
    if len(final.contents[i]) >= 1024:
        drop_ls.append(i)

In [246]:
final.shape

(749, 2)

In [247]:
final1 = final.drop(drop_ls)
final1.shape

(678, 2)

In [248]:
project.save_data(data=final1.to_csv(index=False), file_name='job_label.csv', overwrite=True)

{'asset_id': 'dacfb0f3-52e6-4e07-a913-edd1416ae680',
 'bucket_name': 'hackathon1-donotdelete-pr-swi2lnzkhpnd0w',
 'file_name': 'job_label.csv',
 'message': 'File saved to project storage.'}